<a href="https://colab.research.google.com/github/Kling0012/youtubeSCRs/blob/main/%E9%AB%98%E9%80%9F%E8%AF%AD%E8%A8%80%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "faster-whisper @ https://github.com/nyanta012/faster-whisper/archive/refs/heads/master.tar.gz"
!pip install transformers==4.27.2
!pip install gradio

In [2]:
# whisper large-v2をhugging faceから取得して、ctranslate2でC++に変換&パラメータなどの量子化する
from faster_whisper import WhisperModel
!ct2-transformers-converter --model openai/whisper-large-v2 --output_dir whisper-large-v2-ct2 \
    --copy_files tokenizer.json --quantization float16

model_path = "whisper-large-v2-ct2/"


In [3]:
# Run on GPU with FP16
model = WhisperModel(model_path, device="cuda", compute_type="float16")


**实行完成后在左方侧边栏下载文件**

In [ ]:

from google.colab import files
import os, requests, uuid, json
uploaded = files.upload()
results=[]
def create_srt_file(file_name, results=None, fast_whisper=True):
  results, _ = model.transcribe(file_name,language="ja")
  with open(f"{file_name}.srt", mode="w") as f:
      for index, _dict in enumerate(results):
          if fast_whisper:
            start_time = _dict[0] # start
            end_time = _dict[1] # end
            text = _dict[2] # text
          else:
            start_time = _dict["start"]
            end_time = _dict["end"]
            text = _dict["text"]
          s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
          s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
          s_s, e_s = int(start_time % 60), int(end_time % 60)
          f.write(f'{index+1}\n{s_h:02}:{s_m:02}:{s_s:02},000 --> {e_h:02}:{e_m:02}:{e_s:02},000\n{text}\n\n')

video=uploaded
create_srt_file(video,results=None,fast_whisper=True)

